# 03-getting-started-langfuse

Langfuse の基本事項を試すためのノートブックです。

## 事前準備

In [ ]:
import os
from dotenv import load_dotenv
from operator import itemgetter

from oci.auth.signers import InstancePrincipalsSecurityTokenSigner
from oci.generative_ai_inference.generative_ai_inference_client import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import ChatDetails, OnDemandServingMode, CohereChatRequest

from langchain_cohere import ChatCohere
from langchain_community.llms.oci_generative_ai import OCIGenAI
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.messages import HumanMessage
from langchain_community.llms.oci_generative_ai import OCIGenAI

from langfuse import Langfuse
from langfuse.callback import CallbackHandler
from langfuse.decorators import observe

In [ ]:
load_dotenv()

# Cohere
api_key = os.getenv("COHERE_API_KEY")

# OCI
compartment_id = os.getenv("COMPARTMENT_ID")
endpoint = os.getenv("GENAI_ENDPOINT")

# Langfuse
secret_key = os.getenv("LANGFUSE_SECRET_KEY")
public_key = os.getenv("LANGFUSE_PUBLIC_KEY")
langfuse_host = os.getenv("LANGFUSE_HOST")

langfuse = Langfuse(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host
)
langfuse_handler = CallbackHandler(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host
)

## Tracing

### OCI Generative AI Service(OCI SDK for Python)

In [ ]:
generative_ai_inference_client = GenerativeAiInferenceClient(
    config={},
    signer=InstancePrincipalsSecurityTokenSigner(),
    service_endpoint=endpoint
)

Low-level SDK

In [ ]:
import time
def search(query: str) -> dict:
    time.sleep(2)
    return {"text": "「Oracle Cloud Hangout Cafe」(通称「おちゃかふぇ」/以降、OCHaCafe)は、日本オラクルが主催するコミュニティの1つです。定期的に、開発者・エンジニアに向けたクラウドネイティブな時代に身につけておくべきテクノロジーを深堀する勉強会を開催しています。"}

def generate_text_with_low_level_sdk(query: str, **kwargs):
    trace = langfuse.trace(
        name = "tracing example"
    )
    span = trace.span(
        name = "embedding-search",
        metadata={"database": "mock database"},
        input={"query": query}
    )
    document = search(query)
    span.end(output=document)
    generation = trace.generation(
        name="Text Generation",
        model="cohere.command-r-16k",
        model_parameters={"maxTokens": "512", "temperature": "0.75", "documents": [document["text"]]},
        input=[{"role": "system", "content": "あなたは有能なアシスタントです"}, {"role": "user", "content": query}]
    )
    response = generative_ai_inference_client.chat(
        chat_details=ChatDetails(
            compartment_id=compartment_id,
            serving_mode=OnDemandServingMode(
                model_id="cohere.command-r-16k"
            ),
            chat_request=CohereChatRequest(
                message=query,
                max_tokens=512,
                temperature=0.75,
                documents=[document]
            )
        )
    )
    generation.end(output=response.data.chat_response.text)

generate_text_with_low_level_sdk("OCHaCafeってなんですか？")

Python Decorator

In [ ]:
@observe()
def search_with_decorator(query: str) -> dict:
    time.sleep(2)
    return {"text": "「Oracle Cloud Hangout Cafe」(通称「おちゃかふぇ」/以降、OCHaCafe)は、日本オラクルが主催するコミュニティの1つです。定期的に、開発者・エンジニアに向けたクラウドネイティブな時代に身につけておくべきテクノロジーを深堀する勉強会を開催しています。"}

@observe(as_type="generation")
def generate_text_with_decorator(query: str, **kwargs):
    document = search_with_decorator(query=query)
    res = generative_ai_inference_client.chat(
        chat_details=ChatDetails(
            compartment_id=compartment_id,
            serving_mode=OnDemandServingMode(
                model_id="cohere.command-r-16k"
            ),
            chat_request=CohereChatRequest(
                message=query,
                max_tokens=512,
                documents=[document]
            )
        )
    )
    return res.data

res = generate_text_with_decorator("OCHaCafeってなんですか？")
print(res)

### OCI Generative AI Service(LangChain)

In [ ]:
gen_ai = OCIGenAI(
    auth_type="API_KEY",
    auth_profile="CHICAGO",
    model_id="cohere.command",
    compartment_id=compartment_id,
    model_kwargs={
        "max_tokens": 200
    }
)

res = gen_ai.invoke("OCHaCafeってなんですか？", config={"callbacks": [langfuse_handler]})
print(res)

### Cohere

In [ ]:
chat = ChatCohere(
    model="command-r-plus",
    cohere_api_key=api_key
)

In [ ]:
langfuse_handler = CallbackHandler(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host
)
chat = ChatCohere(
    model="command-r-plus",
    cohere_api_key=api_key
)

prompt1 = ChatPromptTemplate.from_template("{person}はどの街出身ですか？")
prompt2 = ChatPromptTemplate.from_template("{city}はどの国ですか？この質問を{language}で答えてください。")

chain1 = prompt1 | chat | StrOutputParser()
chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | chat
    | StrOutputParser()
)

chain2.invoke({"person": "徳永家康", "language": "スペイン語"}, config={"callbacks": [langfuse_handler]})

## Prompt Management

In [ ]:
langfuse = Langfuse(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host
)

chat = ChatCohere(
    model="command-r-plus",
    cohere_api_key=api_key,
    max_tokens=500
)

chat_prompt = langfuse.get_prompt(name="simple-chat-prompt", type="chat")
# print(chat_prompt.compile(person="徳永家康"))
chat.invoke(chat_prompt.compile(person="徳永家康"), config={"callbacks": [langfuse_handler]})

In [ ]:
langfuse = Langfuse(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host
)

chat = ChatCohere(
    model="command-r-plus",
    cohere_api_key=api_key,
    metadata={
        "max_tokens": 1024
    }
)

chat_prompt = langfuse.get_prompt(name="recipe-chat-prompt", type="chat")
print(chat_prompt.compile(name="カレー"))

chat.invoke(chat_prompt.compile(name="カレー"), config={"callbacks": [langfuse_handler]})

## Score

In [ ]:
langfuse = Langfuse(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host
)

chat = ChatCohere(
    model="command-r-plus",
    cohere_api_key=api_key,
    metadata={
        "max_tokens": 1024
    }
)

chat_prompt = langfuse.get_prompt(name="recipe-chat-prompt", type="chat")
print(chat_prompt.compile(name="カレー"))

chat.invoke(chat_prompt.compile(name="カレー"), config={"callbacks": [langfuse_handler]})

In [ ]:
trace_id = langfuse_handler.get_trace_id()
print("trace_id", trace_id)
trace = langfuse.score(
    trace_id=trace_id,
    value=1,
    name="user-feedback",
    comment="美味しそうなのでOKです"
)

In [ ]:
chat = ChatCohere(model="command-r-plus", cohere_api_key=api_key)

messages = [HumanMessage(content="OCHaCafeとはなんですか？")]
res = chat.invoke(messages, config={"callbacks": [langfuse_handler]})

langfuse = Langfuse(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host
)
langfuse_handler = CallbackHandler(
    secret_key=secret_key,
    public_key=public_key,
    host=langfuse_host
)

In [ ]:
trace_id = langfuse_handler.get_trace_id()
print("trace_id", trace_id)
trace = langfuse.score(
    trace_id=trace_id,
    value=0,
    name="user-feedback",
    comment="OCHaCafeは、お茶をテーマにしたユニークなカフェではありません。"
)

## Testing

In [ ]:
EVAL_TYPES={
    "hallucination": True,
    "conciseness": True,
    "relevance": True,
    "coherence": True,
    "harmfulness": True,
    "maliciousness": True,
    "helpfulness": True,
    "controversiality": True,
    "misogyny": True,
    "criminality": True,
    "insensitivity": True
}

In [ ]:
from langchain.evaluation import load_evaluator
from langchain_community.llms.oci_generative_ai import OCIGenAI

def get_evaluator_for_key(key: str):
    llm = ChatOCIGenAI(
        auth_type="INSTANCE_PRINCIPAL",
        model_id="cohere.command-r-plus",
        compartment_id=compartment_id,
        service_endpoint=endpoint,
        is_stream=False,
        model_kwargs={
            "max_tokens": 500,
            "temperature": 0,
            "top_k": 0,
            "top_p": 0.75,
            "frequency_penalty": 0,
            "presence_penalty": 0,
        }
    )
    return load_evaluator("criteria", criteria=key, llm=llm)

In [ ]:
from langchain.evaluation.criteria import LabeledCriteriaEvalChain

def get_hallucination_eval():
    criteria = {
        "hallucination": (
            "Does this submission contain information"
            "not present in the input or reference?"
        ),
    }
    llm = ChatOCIGenAI(
        auth_type="INSTANCE_PRINCIPAL",
        model_id="cohere.command-r-plus",
        compartment_id=compartment_id,
        service_endpoint=endpoint,
        is_stream=False,
        model_kwargs={
            "max_tokens": 500,
            "temperature": 0,
            "top_k": 0,
            "top_p": 0.75,
            "frequency_penalty": 0,
            "presence_penalty": 0,
        }
    )
    return LabeledCriteriaEvalChain.from_llm(
        llm=llm,
        criteria=criteria,
    )

In [ ]:
def fetch_all_pages(name=None, user_id = None, limit=50):
    page = 1
    all_data = []
 
    while True:
        response = langfuse.get_generations(name=name, limit=limit, user_id=user_id, page=page)
        if not response.data:
            break
 
        all_data.extend(response.data)
        page += 1
 
    return all_data

generations = fetch_all_pages(name="ChatOCIGenAI")

In [ ]:
def execute_eval_and_score():
  for generation in generations:
    criteria = [key for key, value in EVAL_TYPES.items() if value and key != "hallucination"]
    for criterion in criteria:
      eval_result = get_evaluator_for_key(criterion).evaluate_strings(
          prediction=generation.output,
          input=generation.input,
      )
      print(eval_result)
      langfuse.score(name=criterion, trace_id=generation.trace_id, observation_id=generation.id, value=eval_result["score"], comment=eval_result['reasoning'])
 
execute_eval_and_score()

In [ ]:
dataset = langfuse.get_dataset(name="ochat-demo-dataset")
for item in dataset.items:
    handler = item.get_langchain_handler(run_name="ochat-test")
    chat.invoke(input=item.input, config={"callbacks": [handler]})
langfuse.flush()

In [ ]:
import oci

client = oci.object_storage.ObjectStorageClient()
client.create_bucket